In [ ]:
import numpy as np
import pandas as pd
import platform

def check_os():
    os = platform.system()

    if os == "Darwin":
        return "MacOS"
    elif os == "Linux":
        return "Linux"
    else:
        return "Unknown OS"

In [ ]:
operating_system = check_os()

if operating_system == "MacOS":
    root_path = "/Users/johnny/Projects/"
elif operating_system == "Linux":
    root_path = "/home/johnny/Projects/"

In [ ]:
gt_df = pd.read_csv(root_path + "/Master_Thesis_CV/datasets/test_df.csv")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the style of seaborn for better visualization
sns.set_style("whitegrid")

# Create a bar plot for the distribution of "loitering" values
plt.figure(figsize=(10, 6))
sns.countplot(data=gt_df, x='loitering')

plt.title('Distribution of loitering values')
plt.xlabel('Value of loitering')
plt.ylabel('Unique ID\'s counts')

plt.tight_layout()
plt.savefig(root_path + "/Master_Thesis_CV/figures/loitering_distribution.pdf")
plt.show()


In [ ]:
# Load the ground truth and predictions dataframes
gt_df = pd.read_csv(root_path + 'Master_Thesis_CV/datasets/test_df.csv')
predictions_df = pd.read_csv(root_path + 'Master_Thesis_CV/datasets/predictions_NO_IMP.csv')

# Ensure the IDs are aligned for both dataframes
gt_df = gt_df.sort_values(by='id').reset_index(drop=True)
predictions_df = predictions_df.sort_values(by='id').reset_index(drop=True)

# copy loitering column in predictions_df

In [ ]:
# Calculate metrics for each method
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

metrics_data = {}
methods = ["rectangle", "convex_hull", "ellipse", "closed_areas", "no_motion_short_term","no_motion_long_term"]


gt_values = gt_df['loitering'].values

for method in methods:
    pred_values = predictions_df[method].values
    
    tn, fp, fn, tp = confusion_matrix(gt_values, pred_values).ravel()
    
    sensitivity = tp / (tp + fn) if tp + fn > 0 else 0
    specificity = tn / (tn + fp) if tn + fp > 0 else 0
    
    metrics_data[method] = {
        "Accuracy": accuracy_score(gt_values, pred_values),
        "Precision": precision_score(gt_values, pred_values),
        "Recall": recall_score(gt_values, pred_values),
        "F1 Score": f1_score(gt_values, pred_values),
        "Sensitivity": sensitivity,
        "Specificity": specificity,
        "ROC AUC": roc_auc_score(gt_values, pred_values)
    }

# Convert the metrics data to a dataframe for easier visualization
metrics_df = pd.DataFrame(metrics_data).transpose()
metrics_df

In [9]:
from sklearn.metrics import classification_report

gt_values = gt_df['loitering'].values

# Methods to be evaluated
methods = ["rectangle", "convex_hull", "ellipse", "closed_areas", "no_motion_short_term", "no_motion_long_term"]

target_names = ['Not loitering', 'Loitering']

# Initialize a dictionary to hold classification reports for each method
classification_reports = {}

# Evaluate each method
for method in methods:
    pred_values = predictions_df[method].values
    report = classification_report(gt_values, pred_values, output_dict=True, zero_division=0, target_names=target_names)
    classification_reports[method] = report

classification_reports

{'rectangle': {'Not loitering': {'precision': 0.8100953053300388,
   'recall': 0.7226070528967254,
   'f1-score': 0.7638542186719921,
   'support': 3176.0},
  'Loitering': {'precision': 0.20986547085201793,
   'recall': 0.30310880829015546,
   'f1-score': 0.24801271860095386,
   'support': 772.0},
  'accuracy': 0.6405775075987842,
  'macro avg': {'precision': 0.5099803880910284,
   'recall': 0.5128579305934404,
   'f1-score': 0.5059334686364729,
   'support': 3948.0},
  'weighted avg': {'precision': 0.6927251350623002,
   'recall': 0.6405775075987842,
   'f1-score': 0.6629855160238559,
   'support': 3948.0}},
 'convex_hull': {'Not loitering': {'precision': 0.8451964216258265,
   'recall': 0.6841939546599496,
   'f1-score': 0.7562206368540108,
   'support': 3176.0},
  'Loitering': {'precision': 0.2716049382716049,
   'recall': 0.4844559585492228,
   'f1-score': 0.34806886924150765,
   'support': 772.0},
  'accuracy': 0.6451367781155015,
  'macro avg': {'precision': 0.5584006799487158,
 

In [8]:
from prettytable import PrettyTable
for method, report in classification_reports.items():
    print(f"Method: {method}")
    table = PrettyTable()
    table.field_names = ["Class", "Precision", "Recall", "F1-Score", "Support"]
    
    for target_name, metrics in report.items():
        if target_name != 'accuracy' and target_name != 'macro avg' and target_name != 'weighted avg':
            table.add_row([target_name, 
                           round(metrics['precision'], 3), 
                           round(metrics['recall'], 3),
                           round(metrics['f1-score'], 3),
                           metrics['support']])
    
    print(table)
    print(f"Accuracy: {round(report['accuracy'], 2)}")
    print("="*40)

Method: rectangle
+---------------+-----------+--------+----------+---------+
|     Class     | Precision | Recall | F1-Score | Support |
+---------------+-----------+--------+----------+---------+
| Not loitering |    0.81   | 0.723  |  0.764   |  3176.0 |
|   Loitering   |    0.21   | 0.303  |  0.248   |  772.0  |
+---------------+-----------+--------+----------+---------+
Accuracy: 0.64
Method: convex_hull
+---------------+-----------+--------+----------+---------+
|     Class     | Precision | Recall | F1-Score | Support |
+---------------+-----------+--------+----------+---------+
| Not loitering |   0.845   | 0.684  |  0.756   |  3176.0 |
|   Loitering   |   0.272   | 0.484  |  0.348   |  772.0  |
+---------------+-----------+--------+----------+---------+
Accuracy: 0.65
Method: ellipse
+---------------+-----------+--------+----------+---------+
|     Class     | Precision | Recall | F1-Score | Support |
+---------------+-----------+--------+----------+---------+
| Not loitering 

# Majority Voting

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
import numpy as np

# Check if the IDs match between the two dataframes
if not all(gt_df['id'] == predictions_df['id']):
    raise ValueError("The IDs in the ground truth and predictions do not match.")

# Extract the ground truth labels
y_true = gt_df['loitering'].values

# Majority Voting
# For each instance, count the number of times '1' appears across all methods, and decide the final label based on the majority
majority_vote_pred = predictions_df.drop('id', axis=1).apply(lambda row: 1 if np.sum(row) > len(row)/2 else 0, axis=1)

predictions_df['majority_vote'] = majority_vote_pred

# Compute the evaluation metrics for Majority Voting
f1_majority = f1_score(y_true, majority_vote_pred)
precision_majority = precision_score(y_true, majority_vote_pred)
recall_majority = recall_score(y_true, majority_vote_pred)
roc_auc_majority = roc_auc_score(y_true, majority_vote_pred)

f1_majority, precision_majority, recall_majority, roc_auc_majority

print(f"Majority Voting Precision: {round(precision_majority, 3)}")
print(f"Majority Voting Recall: {round(recall_majority, 3)}")
print(f"Majority Voting F1-Score: {round(f1_majority, 3)}")
print(f"Majority Voting ROC AUC: {round(roc_auc_majority, 3)}")


# Weighted voting

In [ ]:
# Calculate the individual F1-scores for each prediction method to use as weights
weights = {}
for method in predictions_df.columns[1:]:
    f1_individual = f1_score(y_true, predictions_df[method])
    weights[method] = f1_individual

# Compute the weighted sum of predictions for each instance
weighted_vote_pred = predictions_df.drop('id', axis=1).apply(lambda row: np.dot(row, list(weights.values())), axis=1)
weighted_vote_final_pred = (weighted_vote_pred >= np.mean(list(weights.values()))).astype(int)

predictions_df['weighted_vote'] = weighted_vote_final_pred

# Compute the evaluation metrics for Weighted Voting
f1_weighted = f1_score(y_true, weighted_vote_final_pred)
precision_weighted = precision_score(y_true, weighted_vote_final_pred)
recall_weighted = recall_score(y_true, weighted_vote_final_pred)
roc_auc_weighted = roc_auc_score(y_true, weighted_vote_final_pred)

weights, f1_weighted, precision_weighted, recall_weighted, roc_auc_weighted

print(f"Weighted Voting Precision: {round(precision_weighted, 3)}")
print(f"Weighted Voting Recall: {round(recall_weighted, 3)}")
print(f"Weighted Voting F1-Score: {round(f1_weighted, 3)}")
print(f"Weighted Voting ROC AUC: {round(roc_auc_weighted, 3)}")


In [ ]:
# save new predictions
predictions_df.to_csv(root_path + 'Master_Thesis_CV/datasets/predictions_2.csv', index=False)

In [ ]:
# Importing additional classifiers that can be used as meta-models
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

# Define base models based on the user's input methods
def implement_stacking(X, y, base_models, meta_models):
    # Splitting the data into training and test sets (30% held out for testing)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Dictionary to store the classification reports for different meta-models
    classification_reports = {}
    
    # Loop through each classifier and perform stacking
    for name, model in meta_models:
        stacking_clf = StackingClassifier(estimators=base_models, final_estimator=model)
        stacking_clf.fit(X_train, y_train)
        y_pred = stacking_clf.predict(X_test)
        report = classification_report(y_test, y_pred)
        classification_reports[name] = report
        
    return classification_reports

base_models = [(method, RandomForestClassifier(n_estimators=100, max_depth=2, random_state=42)) for method in methods]

# List of classifiers to use as meta-models
meta_models = [
    ('Logistic Regression', LogisticRegression()),
    ('Random Forest', RandomForestClassifier()),
    ('Gradient Boosting', GradientBoostingClassifier()),
    ('Support Vector Classifier', SVC(probability=True)),
    ('XGBoost', XGBClassifier(eval_metric='logloss'))
]

# Implement stacking and get classification reports
classification_reports = implement_stacking(predictions_df[methods], gt_values, base_models, meta_models)
display(classification_reports)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

# Prepare the feature matrix and target vector
X = predictions_df.drop('id', axis=1).values

# Initialize the meta-classifier (Logistic Regression)
meta_clf = LogisticRegression()

# Use cross-validation to get the predicted labels
stacking_pred = cross_val_predict(meta_clf, X, y_true, cv=5)

# Compute the evaluation metrics for Stacking with cross-validation
f1_stacking = f1_score(y_true, stacking_pred)
precision_stacking = precision_score(y_true, stacking_pred)
recall_stacking = recall_score(y_true, stacking_pred)
roc_auc_stacking = roc_auc_score(y_true, stacking_pred)

f1_stacking, precision_stacking, recall_stacking, roc_auc_stacking
print(f"Stacking Precision: {round(precision_stacking, 3)}")
print(f"Stacking Recall: {round(recall_stacking, 3)}")
print(f"Stacking F1-Score: {round(f1_stacking, 3)}")
print(f"Stacking ROC AUC: {round(roc_auc_stacking, 3)}")


In [ ]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

# Initialize meta-classifiers
meta_clfs = {
    'Logistic Regression': LogisticRegression(),
    'XGBoost': XGBClassifier(),
    'Random Forest': RandomForestClassifier()
}

# Dictionary to store the evaluation metrics for each meta-classifier
stacking_metrics = {}

# Perform stacking with cross-validation for each meta-classifier
for name, meta_clf in meta_clfs.items():
    stacking_pred = cross_val_predict(meta_clf, X, y_true, cv=5, n_jobs=-1)
    
    # Compute the evaluation metrics
    f1 = f1_score(y_true, stacking_pred)
    precision = precision_score(y_true, stacking_pred)
    recall = recall_score(y_true, stacking_pred)
    roc_auc = roc_auc_score(y_true, stacking_pred)
    
    # Store the metrics
    stacking_metrics[name] = {
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1,
        'ROC_AUC': roc_auc
    }

stacking_metrics


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Dictionary to store the detailed evaluation metrics for each meta-classifier
stacking_detailed_metrics = {}

# Perform stacking with cross-validation for each meta-classifier
for name, meta_clf in meta_clfs.items():
    stacking_pred = cross_val_predict(meta_clf, X, y_true, cv=10, n_jobs=-1)
    
    # Compute the evaluation metrics
    f1 = f1_score(y_true, stacking_pred)
    precision = precision_score(y_true, stacking_pred)
    recall = recall_score(y_true, stacking_pred)
    roc_auc = roc_auc_score(y_true, stacking_pred)
    
    # Class-wise metrics
    class_report = classification_report(y_true, stacking_pred, target_names=['Class 0', 'Class 1'], output_dict=True)
    tn, fp, fn, tp = confusion_matrix(y_true, stacking_pred).ravel()
    
    # Store the metrics
    stacking_metrics[name] = {
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1,
        'ROC_AUC': roc_auc
    }
    
    # Store the detailed metrics
    stacking_detailed_metrics[name] = {
        'Classification Report': class_report,
        'True Negative': tn,
        'False Positive': fp,
        'False Negative': fn,
        'True Positive': tp,
    }

# Display summary metrics
print("Summary Metrics:")
print(stacking_metrics)

# Display detailed metrics
print("\nDetailed Metrics:")
for name, metrics in stacking_detailed_metrics.items():
    print(f"\nFor {name}:")
    print(f"Classification Report: {metrics['Classification Report']}")


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Dictionary to store detailed metrics
detailed_metrics = {}

# Function to calculate class-wise metrics
def get_classwise_metrics(y_true, y_pred):
    class_report = classification_report(y_true, y_pred, output_dict=True)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return {
        'Classification Report': class_report,
        'True Negative': tn,
        'False Positive': fp,
        'False Negative': fn,
        'True Positive': tp
    }

# For Majority Voting
majority_metrics = get_classwise_metrics(y_true, majority_vote_pred)
detailed_metrics['Majority Voting'] = majority_metrics

# For Weighted Voting
weighted_metrics = get_classwise_metrics(y_true, weighted_vote_final_pred)
detailed_metrics['Weighted Voting'] = weighted_metrics

# Display detailed metrics
print("\nDetailed Metrics:")
for name, metrics in detailed_metrics.items():
    print(f"\nFor {name}:")
    print(f"Classification Report: {metrics['Classification Report']}")
    print(f"True Negative: {metrics['True Negative']}")
    print(f"False Positive: {metrics['False Positive']}")
    print(f"False Negative: {metrics['False Negative']}")
    print(f"True Positive: {metrics['True Positive']}")
